如果网页源代码中没有我们需要的数据，说明数据是通过 API 加载的，我们则需要在 Network 中的 XHR 里查找数据。我们可以通过 一个一个查找、根据名称和大小查找、清空请求列表，获取新的请求查找 这三种方法找到数据。

至此，大部分网站已经拦不住你了，你可以爬取你想要的任何数据。除了获取数据之外，爬虫还能帮你自动做一些事情，比如：发微博、转发、评论微博、抢火车票、飞机票、景点门票等等。

刚才说得那些操作都有一个前提——必须有对应的账号。因此，我们的爬虫也就需要登录网站才能进行接下来的操作。今天，我们就来说说爬虫的模拟登录。

登录博客
登录账号，是我们生活中再平常不过的事情了。登录微信、登录微博、登录 12306 等等，我们以前只是输入账号密码，然后点击登录按钮，但你了解这背后发生了什么吗？

登录完成后，我们在右边的请求列表里点击第一条请求（wp-login.php）。右边的 Headers 里我们可以看到请求地址（Request URL）为 https://wpblog.x0y1.com/wp-login.php，请求方式（Request Method）是 POST，状态码为 302。

Tips：302 表示临时页面跳转，所以登录后会跳转到博客首页。

### POST 请求

GET 和 POST 本质上的区别是：

GET 用于获取数据，比如刷微博；
POST 用于提交数据，比如登录微博。

GET 和 POST 形式上的区别是：

GET 的参数显示在请求地址里；
POST 的参数隐藏在 Form Data 里。

In [1]:
import requests

headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}
data = { # From Data
  'log': 'codetime',
  'pwd': 'shanbay520',
  'wp-submit': '登录',
  'redirect_to': 'https://wpblog.x0y1.com',
  'testcookie': '1'
}
requests.post('https://wpblog.x0y1.com/wp-login.php', data=data, headers=headers)

<Response [200]>

在发出 POST 请求之后，服务器会在 Response Headers（响应头）里返回一些关于该请求的信息。有内容的格式、内容大小、过期时间等信息，我们重点关注的是有关登录的信息——set-cookie

set-cookie 的作用是在浏览器中写入 cookie，之后的请求中会带上 cookie 信息。而我们的登录信息就藏在其中，所以在登录后，服务器能判断出我们是否已经登录。

简单地说，cookie 是浏览器储存在用户电脑上的一小段文本文件。该文件里存了加密后的用户信息，过期时间等，且每次请求都会带上 cookie。所以，你登录过某网站后，下次再次打开该网站便不再需要登录。

因为 cookie 有过期时间，因此一段时间之后，cookie 便会失效，需要你再次重新登录，生成新的 cookie。cookie 就像一张通行证，当没有或通行证过期了，就无法通过，需要重新办理通行证才行。

In [2]:
import requests

headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}
data = {
  'log': 'codetime',
  'pwd': 'shanbay520',
  'wp-submit': '登录',
  'redirect_to': 'https://wpblog.x0y1.com',
  'testcookie': '1'
}
r = requests.post('https://wpblog.x0y1.com/wp-login.php', data=data, headers=headers)
print(r.cookies)

<RequestsCookieJar[<Cookie wordpress_logged_in_95afba76913e9e030edc1403a85c976a=codetime%7C1657859865%7CNgTk1nRE47QRPVyOkZUboUTtgV9KT5Z4zlbb7rIwdKe%7Cdc7e6bfae8afe422869d3bce2427c5d0f17f26e431138fcd921fa63df44fc75a for wpblog.x0y1.com/>, <Cookie wordpress_test_cookie=WP+Cookie+check for wpblog.x0y1.com/>, <Cookie wordpress_sec_95afba76913e9e030edc1403a85c976a=codetime%7C1657859865%7CNgTk1nRE47QRPVyOkZUboUTtgV9KT5Z4zlbb7rIwdKe%7C3e80be1d49f97fba79ac0e6811decc9f2e312d8936e7b8d333ce3323b710c9e5 for wpblog.x0y1.com/wp-admin>, <Cookie wordpress_sec_95afba76913e9e030edc1403a85c976a=codetime%7C1657859865%7CNgTk1nRE47QRPVyOkZUboUTtgV9KT5Z4zlbb7rIwdKe%7C3e80be1d49f97fba79ac0e6811decc9f2e312d8936e7b8d333ce3323b710c9e5 for wpblog.x0y1.com/wp-content/plugins>]>


需要注意的是，requests 请求之间的 cookie 不共享。因此，我们需要手动将登录过后的 cookie 传给爬取文章的请求。我先带你看一遍怎么写，请仔细阅读下面代码：

In [3]:
import requests
from bs4 import BeautifulSoup

headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}
# 登录参数
login_data = {
  'log': 'codetime',
  'pwd': 'shanbay520',
  'wp-submit': '登录',
  'redirect_to': 'https://wpblog.x0y1.com',
  'testcookie': '1'
}

# 发请求登录
login_req = requests.post('https://wpblog.x0y1.com/wp-login.php', data=login_data, headers=headers)

# 获取登录后的 cookies
shared_cookies = login_req.cookies

# 将登录后的 cookies 传递给 cookies 参数用于获取文章页面内容
res = requests.get('https://wpblog.x0y1.com/?cat=2', cookies=shared_cookies, headers=headers)

# 解析页面
soup = BeautifulSoup(res.text, 'html.parser')

# 选择所有的代表标题的 a 标签
titles = soup.select('h2.entry-title a')

# 先打印结果看看
for i in titles:
  print(i)

<a href="https://wpblog.x0y1.com/?p=211" rel="bookmark">编程实战 – 电商交易流程分析</a>
<a href="https://wpblog.x0y1.com/?p=199" rel="bookmark">2006年～2019年中国国内生产总值 (GDP) 统计</a>
<a href="https://wpblog.x0y1.com/?p=43" rel="bookmark">理解二进制难倒了很多人？别急，小贝马上教会你</a>
<a href="https://wpblog.x0y1.com/?p=34" rel="bookmark">Python 环境搭建指南</a>
<a href="https://wpblog.x0y1.com/?p=8" rel="bookmark">因为他，Python 成为当下最红编程语言</a>


对于每个 a 标签，我们都需要取出其 href 属性中的链接。BeautifulSoup 解析结果中，每个标签都可以通过 标签.attrs 的方式，将标签属性以字典的格式返回，我们以第一个 a 标签 为例：

In [5]:
title = titles[0]
# 打印标签文本
print(title.text)
# 打印标签属性
print(title.attrs)

编程实战 – 电商交易流程分析
{'href': 'https://wpblog.x0y1.com/?p=211', 'rel': ['bookmark']}


In [ ]:
所以，我们可以借助列表生成式，将所有 titles 中所以标题链接放进一个列表中。

In [6]:
links = [i.attrs['href'] for i in titles]
print(links)

['https://wpblog.x0y1.com/?p=211', 'https://wpblog.x0y1.com/?p=199', 'https://wpblog.x0y1.com/?p=43', 'https://wpblog.x0y1.com/?p=34', 'https://wpblog.x0y1.com/?p=8']


In [7]:
for link in links:
  # 获取文章页面内容
  res_psg = requests.get(link, cookies=shared_cookies, headers=headers)
  # 解析文章页面
  soup_psg = BeautifulSoup(res_psg.text, 'html.parser')
  # 获取文章内容的标签
  content = soup_psg.select('div.entry-content')[0]
  # 打印文章内容
  print(content.text)







环节
人数


浏览网站
9943


加入购物车
3320


生成订单
963


支付订单
646


完成交易
412














年份
绝对额(亿元)   
同比增长(%)   


2006年
219438.5
12.70%


2007年
270092.3
14.20%


2008年
319244.6
9.70%


2009年
348517.7
9.40%


2010年
412119.3
10.60%


2011年
487940.2
9.50%


2012年
538580
7.90%


2013年
592963.2
7.80%


2014年
643563.1
7.30%


2015年
688858.2
6.90%


2016年
746395.1
6.70%


2017年
832035.9
6.90%


2018年
919281.1
6.60%


2019年
990865.1
6.10%











提到进位制，大家应该都对十进制比较熟悉，因为生活中大都使用十进制的，我们从小学就开始学习十进制的四则运算。既然人类对十进制这么熟悉，为什么计算机不和人类使用一样的进制呢？






二进制的表示


计算机底层由一堆电子电路组成，我们来看一个最简单的电路例子：







你可能已经猜到上面电路的功能了：



开关闭合时，灯泡亮
开关断开时，灯泡灭



我们可以用开关闭合代表 1，开关断开代表 0。这就是一个简单的二进制电路。


与此类似，磁盘其实是由很多小磁铁组成的，磁铁的 N 极表示 1，S 极表示 0。所以每个小磁铁和开关一样，也是有 0 和 1 两种状态。


早期的元器件只能表示这两种状态（比如开和关、 N 和 S 等），这就是为什么计算机用的是二进制的原因了。你想想，如果计算机用十进制的话，这个电子元器件得有十种状态，该多么复杂呢！


二进制的位和字节


不过仅用一位数能表示的数字有限，比如二进制是 0-1，十进制是 0-9，十六进制是 0-15。一位数字当然满足不了我们的需求，如果计算机只能处理 0-1 的范围，人类只能处理 0-9 的范围，一个磁盘就只能存储 0 和 1 这两个数字，那么社会发展可能早就停滞不前了吧！


聪明的人类选择用更多的位数来计数，十进制里我们

前面说过，我们可以将登录后获得的 cookie 传递给后续的请求，以保持登录状态。但这样每次都要传递 cookie 很是麻烦，有没有什么方法可以让登录状态在多个请求之间共享呢？当然有！那就是 session。

### session
因为 HTTP 是无状态的，在一次请求、响应结束过后，连接就断开了。再次发起请求时，之前的状态全都丢失了，服务器也不再“认识你”。

有了 cookie 之后，我们可以将一些信息存到其中，比如用户身份信息等。但因为 cookie 容量有限，只有 4KB。因此，不可能将所有的用户信息都存到里面。这时候，session 就出现了。

session 相当于在服务器上建立的一份用户档案，cookie 中只要存储用户的身份信息，服务器通过身份信息在 session 中查询用户的其他信息。这样一来，我们的所有操作都会被保留。比如我们添加到购物车的商品，重新打开页面后仍会被保留。

文档说，我们可以通过 requests.Session() 创建一个 session，注意 S 要大写。然后我们就可以像使用 requests 一样使用 session 对象了，get()、post() 等方法统统都有，只需将原先的 requests 替换成我们创建的 session 即可。

有了 session，多个请求之间就可以共享 cookie 了，后续请求便不再需要传 cookies 参数。

除了 cookies 参数每次都要传很麻烦，headers 参数每次都要传也很麻烦。如果想要共享 headers 的话，可以像下面这样写：

In [8]:
import requests

session = requests.Session()
headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}
# 设置 session 的全局 headers
session.headers.update(headers)
# 默认使用全局的 headers
session.get('https://wpblog.x0y1.com')
# 自定义 headers
custom_headers = { 'referer': 'https://wpblog.x0y1.com' }
session.get('https://wpblog.x0y1.com', headers=custom_headers)
# 既有全局的 user-agent 也有自定义的 referer

<Response [200]>

我们可以通过 session.headers.update() 方法来更新全局的 headers，通过该 session 发送的请求都会使用我们设置的全局 headers。

当全局 headers 不满足我们的需求时，也可以给某个请求单独设置 headers。这时，该请求将同时拥有全局和单独设置的 headers。如果两个 headers 里的字段重复，会优先使用单独设置的 headers 字段的值。

In [9]:
import requests
from bs4 import BeautifulSoup

headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}
# 登录参数
login_data = {
  'log': 'codetime',
  'pwd': 'shanbay520',
  'wp-submit': '登录',
  'redirect_to': 'https://wpblog.x0y1.com',
  'testcookie': '1'
}

session = requests.Session()
session.headers.update(headers)
# 使用 session 登录
login_req = session.post('https://wpblog.x0y1.com/wp-login.php', data=login_data)
# 使用 session 获得 Python 分类文章
comment_req = session.get('https://wpblog.x0y1.com/?cat=2')

# 解析页面
soup = BeautifulSoup(comment_req.text, 'html.parser')
# 选择所有的代表标题的 a 标签
titles = soup.select('h2.entry-title a')
# 获取四篇文章的链接
links = [i.attrs['href'] for i in titles]

for link in links:
  # 获取文章页面内容
  res_psg = session.get(link)
  # 解析文章页面
  soup_psg = BeautifulSoup(res_psg.text, 'html.parser')
  # 获取文章内容的标签
  content = soup_psg.select('div.entry-content')[0]
  # 打印文章内容
  print(content.text)







环节
人数


浏览网站
9943


加入购物车
3320


生成订单
963


支付订单
646


完成交易
412














年份
绝对额(亿元)   
同比增长(%)   


2006年
219438.5
12.70%


2007年
270092.3
14.20%


2008年
319244.6
9.70%


2009年
348517.7
9.40%


2010年
412119.3
10.60%


2011年
487940.2
9.50%


2012年
538580
7.90%


2013年
592963.2
7.80%


2014年
643563.1
7.30%


2015年
688858.2
6.90%


2016年
746395.1
6.70%


2017年
832035.9
6.90%


2018年
919281.1
6.60%


2019年
990865.1
6.10%











提到进位制，大家应该都对十进制比较熟悉，因为生活中大都使用十进制的，我们从小学就开始学习十进制的四则运算。既然人类对十进制这么熟悉，为什么计算机不和人类使用一样的进制呢？






二进制的表示


计算机底层由一堆电子电路组成，我们来看一个最简单的电路例子：







你可能已经猜到上面电路的功能了：



开关闭合时，灯泡亮
开关断开时，灯泡灭



我们可以用开关闭合代表 1，开关断开代表 0。这就是一个简单的二进制电路。


与此类似，磁盘其实是由很多小磁铁组成的，磁铁的 N 极表示 1，S 极表示 0。所以每个小磁铁和开关一样，也是有 0 和 1 两种状态。


早期的元器件只能表示这两种状态（比如开和关、 N 和 S 等），这就是为什么计算机用的是二进制的原因了。你想想，如果计算机用十进制的话，这个电子元器件得有十种状态，该多么复杂呢！


二进制的位和字节


不过仅用一位数能表示的数字有限，比如二进制是 0-1，十进制是 0-9，十六进制是 0-15。一位数字当然满足不了我们的需求，如果计算机只能处理 0-1 的范围，人类只能处理 0-9 的范围，一个磁盘就只能存储 0 和 1 这两个数字，那么社会发展可能早就停滞不前了吧！


聪明的人类选择用更多的位数来计数，十进制里我们

怎么样？相比之前，我们的代码又简洁了很多！并且在向同一个网站发送多个请求时，使用 session 可以复用 TCP 连接，从而带来显著的性能提升！

为了实现同样的功能，我们可以写出不同的代码。每个人写得代码可能都千差万别。我们在写代码时，除了实现想要的功能之外，我们还应该追求写出“优雅简洁”和高性能的代码。

当然，对于刚入门编程的你来说，正确地写出代码、实现功能更加重要。但要想成为厉害的 coder，还是需要“有点追求”的~